In [1]:
import numpy as np
import pandas as pd
import scipy.stats as sts

In [2]:
cars_df = pd.read_csv('/Users/mmenadzhiev/Desktop/Проект по анализу данных/cars_df_final.csv')

In [3]:
cars_df['Accidents or damage'] = (cars_df['Accidents or damage'] == 'At least 1 accident or damage reported')
cars_df['1-owner vehicle'] = (cars_df['1-owner vehicle'] == 'Yes')
cars_df['Personal use only'] = (cars_df['Personal use only'] == 'Yes')

In [4]:
print(cars_df.shape)
cars_df.head()

(2542, 17)


,Brand,Year,Exterior color,Interior color,Drivetrain,Fuel type,Capacity,Configuration,Valves,Accidents or damage,1-owner vehicle,Personal use only,MPG_min,MPG_max,MPG_mean,Mileage,Price
0,Hyundai,2022,Red,Black,Front-wheel Drive,Gasoline,2.5,I4,16.0,True,True,True,22.000000,28.000000,25.000000,13256.0,38988.0
1,Lexus,2016,Black,Black,All-wheel Drive,Gasoline,3.5,V6,24.0,True,False,True,19.000000,26.000000,22.500000,100067.0,19747.0
2,Chevrolet,2017,White,Black,Front-wheel Drive,Hybrid,1.5,I4,16.0,True,True,False,20.592777,27.239548,23.916163,39032.0,25999.0
3,Mercedes-Benz,2022,White,Other,All-wheel Drive,Gasoline,3.0,I6,24.0,False,True,False,20.592777,27.239548,23.916163,10901.0,96750.0
4,Land Rover,2020,Blue,Other,Four-wheel Drive,Gasoline,3.0,I6,24.0,False,True,True,17.000000,22.000000,19.500000,24946.0,61530.0


# Гипотезы

__Заметка:__ Поверим, что цены автомобилей независимы.

__Гипотезы:__

1) Наличие ДТП снижает цену в среднем на x % (Автомобили, поучаствовавшие в ДТП, в среднем стоят на x% дешевле): критерий ...

   Дисперсия и мат. ожидание неизвестны

2) Американские и европейские автомобили в среднем стоят одинаково

3) 0,9 автомобилей ездят на бензине

4) Американские автомобили в среднем расходуют на 20% больше бензина, чем европейские

5) Большинство автомобилей имеет передний привод (p > 0.5)

6) На современных автомобилях (новее 2015 года включительно) средний объем двигателя больше, чем у старых



---

### **5)** Большинство автомобилей имеет передний привод (p > 0.5)

Пусть $p$ - доля автомобилей, ездящих на бензине. Тогда 

$H_{0}: p = 0.5$

$H_{1}: p < 0.5$

In [5]:
alpha = 0.05

In [6]:
z_crit = sts.norm.ppf(1 - alpha)
z_crit

1.6448536269514722

In [7]:
p_hat, p = (cars_df['Drivetrain'] == 'Front-wheel Drive').mean(), 0.5
n_obs = cars_df.shape[0]

In [8]:
z_obs = (p_hat - p) / np.sqrt(p_hat * (1 - p_hat) / n_obs)
z_obs

-30.31869999086294

In [9]:
if z_obs < -z_crit:
    print("H_0 отвергается")
else:
    print("H_0 не отвергается")

H_0 отвергается


### **3)** 90% автомобилей ездят на бензине

Пусть $p$ - доля автомобилей, ездящих на бензине. Тогда 

$H_{0}: p = 0.9$

$H_{1}: p \neq 0.9$

Заметим, что $\large \frac {\hat p - p} {\sqrt{\frac{\hat p (1 - \hat p)} {n}}}$ $\sim N(0, 1)$

In [10]:
n = cars_df.shape[0]
p, p_hat = 0.9, cars_df[cars_df['Fuel type'] == 'Gasoline'].shape[0] / n
z_obs = (p_hat - p) / np.sqrt(p_hat * (1 - p_hat) / n)
z_obs

-3.9342925480047013

In [11]:
p_value = 2 * (1 - sts.norm.cdf(np.abs(z_obs)))

In [12]:
p_value

8.344210198640845e-05

In [13]:
alpha = 0.05

In [14]:
p_value < alpha

True

**Вывод:** Нулевая гипотеза отвергается.

---

### **1)** Наличие ДТП снижает цену на x % (Автомобили, поучаствовавшие в ДТП, в среднем стоят на x% дешевле): критерий ...


__Заметка:__ В данном случае мы разбиваем всю выборку на две группы: автомобили, попавшие и не попавшие в ДТП. Достоверное распределение цены автомобилей неизвестно, но наблюдений достаточно много, чтобы воспользоваться ЦПТ. Также неизвестна и дисперсия, поэтому проверим гипотезу об их равенстве.

$H_{0}: \sigma_{x} = \sigma_{y}$

$H_{1}: \sigma_{x} \neq \sigma_{y}$

In [15]:
alpha = 0.05

In [16]:
damaged_cars_df = cars_df[cars_df['Accidents or damage']]
dc_mean_price = damaged_cars_df['Price'].mean()
print(damaged_cars_df.shape)
damaged_cars_df.head()


intact_cars_df = cars_df[~cars_df['Accidents or damage']]
ic_mean_price = intact_cars_df['Price'].mean()
print(intact_cars_df.shape)
intact_cars_df.head()

(510, 17)
(2032, 17)


,Brand,Year,Exterior color,Interior color,Drivetrain,Fuel type,Capacity,Configuration,Valves,Accidents or damage,1-owner vehicle,Personal use only,MPG_min,MPG_max,MPG_mean,Mileage,Price
3,Mercedes-Benz,2022,White,Other,All-wheel Drive,Gasoline,3.0,I6,24.0,False,True,False,20.592777,27.239548,23.916163,10901.0,96750.0
4,Land Rover,2020,Blue,Other,Four-wheel Drive,Gasoline,3.0,I6,24.0,False,True,True,17.000000,22.000000,19.500000,24946.0,61530.0
5,Lexus,2018,Silver/Gray,Other,Rear-wheel Drive,Gasoline,3.5,V6,24.0,False,False,True,19.000000,30.000000,24.500000,33005.0,49878.0
6,Chevrolet,2006,Silver/Gray,Silver/Gray,Front-wheel Drive,E85 Flex Fuel,3.5,V6,12.0,False,True,False,21.000000,31.000000,26.000000,168810.0,5909.0
7,INFINITI,2020,White,White,Rear-wheel Drive,Gasoline,3.0,V6,24.0,False,True,True,20.000000,26.000000,23.000000,34487.0,38594.0


In [17]:
n_x, n_y = damaged_cars_df.shape[0], intact_cars_df.shape[0]
l_tail = sts.f.ppf(alpha / 2, n_x - 1, n_y - 1)
r_tail = sts.f.ppf(1 - alpha / 2, n_x - 1, n_y - 1)
l_tail, r_tail

(0.8690236736851707, 1.144130115154565)

In [18]:
s_hat_x = np.var(damaged_cars_df['Price'], ddof=1)
s_hat_y = np.var(intact_cars_df['Price'], ddof=1)
f_obs = s_hat_x / s_hat_y
f_obs

0.03570309917715467

In [19]:
if f_obs < l_tail or f_obs > r_tail:
    print(f'H_0 отвергается')
else:
    print(f'H_0 не отверграется')

H_0 отвергается


---

### **2)** Американские и европейские автомобили в среднем стоят одинаково

$H_{0}:$ $\mu_{x} = \mu_{y}$

$H_{1}:$ $\mu_{x} \neq \mu_{y}$

Воспользуемся z-тестом.

In [67]:
cars_df['Brand'].unique()

array(['Hyundai', 'Lexus', 'Chevrolet', 'Mercedes-Benz', 'Land Rover',
       'INFINITI', 'Chrysler', 'Mazda', 'GMC', 'Toyota', 'Cadillac',
       'Genesis', 'Ford', 'Buick', 'Jeep', 'Audi', 'Lincoln', 'BMW',
       'RAM', 'Kia', 'Lamborghini', 'Acura', 'Nissan', 'McLaren',
       'Porsche', 'Honda', 'Volkswagen', 'Pontiac', 'Dodge', 'Mitsubishi',
       'Volvo', 'FIAT', 'MINI', 'Jaguar', 'Aston Martin', 'Maserati',
       'Hummer', 'Bentley', 'Alfa Romeo', 'Plymouth', 'Ferrari', 'Scion',
       'Lotus', 'Suzuki', 'Rolls-Royce', 'Saab'], dtype=object)

In [68]:
# Получим список американских машин. В дальнейшем можно создать новый бинарный признак о стране происхождения.
american_cars = ['Chevrolet', 'Chrysler', 'GMC', 'Cadillac',
                 'Ford', 'Buick', 'Jeep', 'Lincoln', 'RAM',
                 'Pontiac', 'Dodge', 'Hummer', 'Plymouth']

european_cars = ['Mercedes-Benz', 'Audi', 'BMW', 'Lamborghini', 'Porsche', 'Volkswagen',
                 'Volvo', 'FIAT', 'MINI', 'Maserati', 'Alfa Romeo', 'Ferrari', 'Saab']

british_cars = ['Land Rover', 'McLaren', 'Jaguar', 'Aston Martin', 'Bentley', 'Lotus', 'Rolls-Royce']

asian_cars = ['Hyundai', 'Lexus', 'INFINITI', 'Mazda', 'Toyota', 'Genesis', 'Kia',
              'Acura', 'Nissan', 'Honda', 'Mitsubishi', 'Scion', 'Suzuki']

In [69]:
alpha = 0.05

In [70]:
american_cond = np.vectorize(lambda x: x in american_cars)(cars_df['Brand'])
american_cars_df = cars_df[american_cond]
american_cars_df.head()

,Brand,Year,Exterior color,Interior color,Drivetrain,Fuel type,Capacity,Configuration,Valves,Accidents or damage,1-owner vehicle,Personal use only,MPG_min,MPG_max,MPG_mean,Mileage,Price
2,Chevrolet,2017,White,Black,Front-wheel Drive,Hybrid,1.5,I4,16.0,True,True,False,20.592777,27.239548,23.916163,39032.0,25999.0
6,Chevrolet,2006,Silver/Gray,Silver/Gray,Front-wheel Drive,E85 Flex Fuel,3.5,V6,12.0,False,True,False,21.000000,31.000000,26.000000,168810.0,5909.0
8,Chrysler,2016,Black,Black,All-wheel Drive,Gasoline,3.6,V6,24.0,True,False,True,18.000000,27.000000,22.500000,117793.0,16900.0
11,GMC,2018,Metallic,Black,Four-wheel Drive,Gasoline,6.2,V8,16.0,False,False,False,14.000000,22.000000,18.000000,99458.0,36450.0
14,Chevrolet,2022,Metallic,Black,Rear-wheel Drive,Gasoline,3.6,V6,24.0,False,True,True,25.000000,27.239548,26.119774,4600.0,34790.0


In [71]:
european_cond = np.vectorize(lambda x: x in european_cars)(cars_df['Brand'])
european_cars_df = cars_df[european_cond]
#european_cars_df.loc[:, ['Price']] = european_cars_df['Price'] # / 6.785 Получается, что, мб, европейские автомобили в 6.785 раз дороже
european_cars_df.head()

,Brand,Year,Exterior color,Interior color,Drivetrain,Fuel type,Capacity,Configuration,Valves,Accidents or damage,1-owner vehicle,Personal use only,MPG_min,MPG_max,MPG_mean,Mileage,Price
3,Mercedes-Benz,2022,White,Other,All-wheel Drive,Gasoline,3.0,I6,24.0,False,True,False,20.592777,27.239548,23.916163,10901.0,96750.0
10,Mercedes-Benz,2020,Black,Black,All-wheel Drive,Gasoline,2.0,I4,16.0,False,True,False,21.000000,28.000000,24.500000,62059.0,28888.0
26,Audi,2017,Silver/Gray,Black,All-wheel Drive,Gasoline,2.0,I4,16.0,True,False,False,23.000000,27.000000,25.000000,74672.0,32645.0
28,Audi,2019,White,Silver/Gray,All-wheel Drive,Gasoline,2.0,I4,16.0,False,True,True,22.000000,29.000000,25.500000,23768.0,38286.0
40,BMW,2019,Black,Other,All-wheel Drive,Gasoline,3.0,I6,24.0,True,True,True,21.000000,29.000000,25.000000,30231.0,37414.0


In [72]:
X = american_cars_df['Price']
Y = european_cars_df['Price']

X_mean = X.mean()
Y_mean = Y.mean()

X_var_hat = np.var(X, ddof=1)
Y_var_hat = np.var(Y, ddof=1)

n_x, n_y = X.shape[0], Y.shape[0]

d = (X_var_hat / n_x + Y_var_hat / n_y)**2 / (
    X_var_hat**2 / (n_x**2 * (n_x - 1)) + Y_var_hat**2 / (n_y**2 * (n_y - 1)))

In [73]:
t_obs = (X_mean - Y_mean) / np.sqrt(X_var_hat / n_x + Y_var_hat / n_y)
t_obs

-3.0812372919188302

In [74]:
t_crit = sts.t.ppf(1 - alpha / 2, df=d)
t_crit

1.963131963244552

**Вывод:** $H_{0}$ отвергается